# Intro

In [1]:
%matplotlib widget
import starepandas
import geopandas
import modreaders
import matplotlib.pyplot as plt
import pickle
import spires
import numpy
import pandas
import scipy.io
import pystare
import time
import shapely.wkb
import postgis
import sqlalchemy
import contextily
import multiprocessing
import glob
import copy
import dask
import logging

In [2]:
starepandas.__version__

'0.6.6'

In [3]:
dask.config.set({'temporary-directory': '/tablespace/dask'})
dask.config.set({'distributed.comm.timeouts.tcp': '1200s'})
dask.config.set({'distributed.comm.timeouts.connect': '1200s'})
dask.config.get('distributed.comm.timeouts')

{'tcp': '1200s', 'connect': '1200s'}

In [4]:
user = 'spires_rw'
pwd = 'spiresrw'
host= 'localhost'
db_name = 'spires'
uri = f'postgresql+psycopg2://{user}:{pwd}@{host}/{db_name}'

engine = sqlalchemy.create_engine(uri)

In [5]:
mod09_bandnames = ['500m Surface Reflectance Band 1', 
                   '500m Surface Reflectance Band 2', 
                   '500m Surface Reflectance Band 3', 
                   '500m Surface Reflectance Band 4', 
                   '500m Surface Reflectance Band 5', 
                   '500m Surface Reflectance Band 6', 
                   '500m Surface Reflectance Band 7']

mod09ga_bandnames = ['sur_refl_b01_1', 
                     'sur_refl_b02_1', 
                     'sur_refl_b03_1', 
                     'sur_refl_b04_1', 
                     'sur_refl_b05_1', 
                     'sur_refl_b06_1', 
                     'sur_refl_b07_1']

vnp09_bandnames = ['750m Surface Reflectance Band M5', 
                   '750m Surface Reflectance Band M7',
                   '750m Surface Reflectance Band M3',
                   '750m Surface Reflectance Band M4',
                   '750m Surface Reflectance Band M8',
                   '750m Surface Reflectance Band M10',
                   '750m Surface Reflectance Band M11']

# Loading Data

## Loading ROI

In [6]:
with open('data/roi_4326_huge.pickle', 'rb') as f:
    roi = pickle.load(f)
roi_sids = roi.sids.iloc[0]

/tmp/ipykernel_3247591/2809272951.py:2: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  roi = pickle.load(f)


## Loading Cells

In [7]:
cells = postgis.read('cells', con=engine)

In [8]:
cells = starepandas.STAREDataFrame(cells, sids='centers_sids')
intersects = cells.to_stare_level(17, clear_to_level=True).stare_intersects(roi_sids)
cells = cells[intersects]
cells.reset_index(inplace=True, drop=True)

## Loading Worldview
(Subsetted worldview binary snowmap) 

In [9]:
with open('/tablespace/spires/binary_snowmaps/2017-12-11_clippedDF_huge.pickle', 'rb') as f:
    worldview_full = pickle.load(f)
worldview_full = worldview_full.replace(2, 0)

In [10]:
worldview_19_groups = worldview_full.to_stare_level(19, clear_to_level=True).groupby('sids')
worldview_full['sids_19'] = sids
worldview_19_groups = worldview_full.groupby('sids_19')
worldview_19 = worldview_19_groups['snow'].sum().astype(float) / worldview_19_groups['snow'].count().astype(float)
worldview_19 = worldview_19.reset_index(drop=False)
worldview_19.rename(columns={'sids_19': 'sids'}, inplace=True)
worldview_19 = worldview_19.set_sids('sids')
#worldview_19.add_trixels(n_partitions=6400, inplace=True)

NameError: name 'sids' is not defined

In [ ]:
worldview_19.info(memory_usage='deep')

In [ ]:
worldview_18_groups = worldview_19.to_stare_level(18, clear_to_level=True).groupby('sids')
worldview_18 =  worldview_18_groups['snow'].sum().astype(float) / worldview_18_groups['snow'].count().astype(float)
worldview_18 = worldview_18.reset_index(drop=False)
worldview_18 = worldview_18.set_sids('sids')

In [ ]:
worldview_14_groups = worldview_19.to_stare_level(14, clear_to_level=True).groupby('sids')
worldview_14 =  worldview_14_groups['snow'].sum().astype(float) / worldview_14_groups['snow'].count().astype(float)
worldview_14 = worldview_14.reset_index(drop=False)
worldview_14 = worldview_14.set_sids('sids')
worldview_14.add_trixels(n_partitions=320, inplace=True)
worldview_14.set_geometry('trixels').to_file('worldview_trixels.gpkg', driver='GPKG')

## Loading MOD09GA (Gridded)

In [ ]:
import importlib
importlib.reload(modreaders)

In [ ]:
# Huge
x_min = 1294
x_max = 1430
y_min = 532
y_max = 619

bbox = (x_min, x_max, y_min, y_max)

In [ ]:
file_paths = ['/tablespace/spires/mod09ga/MOD09GA.A2017341.h08v05.006.2017345192347.hdf',  # 2017-12-08
              '/tablespace/spires/mod09ga/MOD09GA.A2017342.h08v05.006.2017344025638.hdf', # 2017-12-08
              '/tablespace/spires/mod09ga/MOD09GA.A2017344.h08v05.006.2017346030233.hdf', # 2017-12-10
              '/tablespace/spires/mod09ga/MOD09GA.A2017345.h08v05.006.2017347024314.hdf', # 2017-12-11
              '/tablespace/spires/mod09ga/MOD09GA.A2017346.h08v05.006.2017348030115.hdf', # 2017-12-12
              '/tablespace/spires/mod09ga/MOD09GA.A2017347.h08v05.006.2017349025015.hdf'] # 2017-12-13

mod09ga = []
for file_path in file_paths:
    mod09ga.append(modreaders.read_mod09ga(file_path, bbox, sidecar_path='data/h08v05_stare.nc'))
    
mod09ga = pandas.concat(mod09ga)
mod09ga.reset_index(drop=True, inplace=True)

mod09ga = mod09ga[mod09ga.to_stare_level(17, clear_to_level=True).stare_intersects(roi_sids, n_partitions=64)]
mod09ga = mod09ga.add_trixels(n_partitions=6400, num_workers=64)
mod09ga['points']= geopandas.points_from_xy(mod09ga.lon, mod09ga.lat, crs=4326)

## Loading MOD09 data

In [ ]:
def add_ndvi(df):
    ndvi = (df['500m Surface Reflectance Band 2'] - df['500m Surface Reflectance Band 1'])/(df['500m Surface Reflectance Band 2'] + df['500m Surface Reflectance Band 1'])
    df['ndvi'] = ndvi
    
def add_ndsi(df):
    ndsi = (df['500m Surface Reflectance Band 4'] - df['500m Surface Reflectance Band 6'])/(df['500m Surface Reflectance Band 4'] + df['500m Surface Reflectance Band 6'])
    df['ndsi'] = ndsi

In [ ]:
with open('/tablespace/spires/mod09.pickle', 'rb') as f:
    mod09 = pickle.load(f)

add_ndsi(mod09)
add_ndvi(mod09)
mod09 = mod09.rename(columns={'x': 'scan_pos', 'y': 'track_pos'})
mod09['sids'] = mod09['sids'].astype('Int64')

In [ ]:
mod09 = mod09[mod09.modland_qa==0]
mod09.reset_index(inplace=True, drop=True)

In [ ]:
mod09.info()

### Subsetting to Day

In [ ]:
mod09.set_index('ts_start', inplace=True)
mod09.sort_index(inplace=True)
#mod09_day = copy.copy(pandas.concat([mod09.loc['2017-12-07':'2017-12-08'], mod09.loc['2017-12-10':'2017-12-12']]))
mod09_day = copy.copy(pandas.concat([mod09.loc['2017-12-07':'2017-12-13']]))
mod09.reset_index(inplace=True)
mod09_day.reset_index(inplace=True)

### Add ellipse and circles

In [ ]:
%%time
ellipse_sids = starepandas.io.granules.modis.make_ellipse_sids(mod09_day, n_partitions=1024, level=19, crs=3310, modis_resolution=500)
mod09_day['ellipse_sids'] = ellipse_sids

r_circle = 500
r_earth = 6378137
phi = r_circle /2/3.14/r_earth*360

circle_sids = starepandas.make_circular_sids(mod09_day, level=19, diameter=phi, n_partitions=1024)
mod09_day['circle_sids'] = circle_sids

In [ ]:
mod09_day['circle_trixels'] = mod09_day.set_sids('circle_sids').make_trixels(n_partitions=1024)
mod09_day['ellipse_trixels'] = mod09_day.set_sids('ellipse_sids').make_trixels(n_partitions=1024)
mod09_day[['ts_start', 'ellipse_trixels']].set_geometry('ellipse_trixels').to_file('ellipses.gpkg', driver='GPKG')
#mod09_day.set_trixels('ellipse_trixels').plot(linewidth=0.1)

## Loading VNP09 Data

In [ ]:
with open('/tablespace/spires/vnp09.pickle', 'rb') as f:
    vnp09 = pickle.load(f)
    
vnp09.sort_values('ts_start', inplace=True)
vnp09.set_index('ts_start', inplace=True)
vnp09_day = copy.copy(vnp09.loc['2017-12-07':'2017-12-13'])
vnp09_day.reset_index(inplace=True)
vnp09.reset_index(inplace=True)

def add_ndvi(df):
    ndvi = (df['750m Surface Reflectance Band M7'] - df['750m Surface Reflectance Band M5'])/(df['750m Surface Reflectance Band M7'] + df['750m Surface Reflectance Band M5'])
    df['ndvi'] = ndvi
    
def add_ndsi(df):
    ndsi = (df['750m Surface Reflectance Band M4'] - df['750m Surface Reflectance Band M10'])/(df['750m Surface Reflectance Band M4'] + df['750m Surface Reflectance Band M10'])
    df['ndsi'] = ndsi

add_ndsi(vnp09)
add_ndvi(vnp09)

In [ ]:
r_circle = 750
r_earth = 6378137
phi = r_circle /2/3.14/r_earth*360

circle_sids = starepandas.make_circular_sids(vnp09_day, level=19, diameter=phi, n_partitions=1024)
vnp09_day['circle_sids'] = circle_sids

In [ ]:
pts = geopandas.points_from_xy(vnp09_day.lon, vnp09_day.lat, crs=4326)
vnp09_day['center_point'] = pts

## Verify Overlap

In [ ]:
fig = plt.figure(figsize=(5, 5), dpi=100)
ax = fig.add_subplot()

cells.set_geometry('polygons_4326').exterior.plot(ax=ax, linewidth=0.5)
mod09[0:10000].set_geometry('center_point').plot(ax=ax, trixels=False, markersize=0.1, color='red')
vnp09_day[0:10000].set_geometry('center_point').plot(ax=ax, trixels=False, markersize=0.1, color='green')
worldview_14.plot(ax=ax, trixels=True, linewidth=0.5, color='black')
mod09ga.set_geometry('trixels').plot(ax=ax, color='magenta', linewidth=1, zorder=10)
roi.plot(ax=ax, color='green', linewidth=0.5)

contextily.add_basemap(ax, source=contextily.providers.OpenStreetMap.Mapnik, crs=4326)

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=100)
ax = fig.add_subplot()

max_elipse = copy.copy(mod09_day[mod09_day['SensorZenith'] == mod09_day['SensorZenith'].max()][0:2])
max_elipse.set_sids('circle_sids').add_trixels().plot(ax=ax, linewidth=0.1, color='magenta')
max_elipse.set_sids('ellipse_sids').add_trixels().plot(ax=ax, linewidth=0.1)
max_elipse.set_sids('sids14').add_trixels().plot(ax=ax, color='green')
max_elipse.set_sids('sids14').add_trixels().plot(ax=ax, color='green')
max_elipse.set_sids('sids15').add_trixels().plot(ax=ax, color='green')
max_elipse.set_sids('sids16').add_trixels().plot(ax=ax, color='green')
max_elipse.set_sids('sids17').add_trixels().plot(ax=ax, color='green')
max_elipse.set_sids('sids18').add_trixels().plot(ax=ax, color='green')
max_elipse.add_trixels().plot(ax=ax, color='red', zorder=1)
max_elipse.center_point.plot(ax=ax, color='red', zorder=1)
contextily.add_basemap(ax, source=contextily.providers.OpenStreetMap.Mapnik, crs=4326)

In [ ]:
total_terra_time = (mod09['ts_start'].max() - mod09['ts_start'].min()).days
total_terra_time

## Load lut

In [ ]:
lut = spires.load_lut('data/LUT_MODIS.mat')

## Load R0

In [ ]:
r0_files = sorted(glob.glob('/tablespace/spires/r0/r0_*.pickle'))

r0 = {}

for file_name in r0_files:    
    ds_name = file_name.split('/')[-1].split('r0_')[1].split('.pickle')[0]
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
    r0[ds_name] = data

In [ ]:
r0_files = sorted(glob.glob('/tablespace/spires/r0/vnp/r0_*.pickle'))

r0_vnp = {}

for file_name in r0_files:    
    ds_name = file_name.split('/')[-1].split('r0_')[1].split('.pickle')[0]
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
    r0_vnp[ds_name] = data

# Run spires

## Merging Frames

In [ ]:
mod09ga_small = mod09ga[['ts_start', 'x', 'y', 'cloud_shadow', 'cloud_internal', 'modland_qa', 'cloud'] + mod09ga_bandnames]
mod09_small = mod09_day[mod09_bandnames + ['scan_pos','SolarZenith', 'SensorAzimuth', 'circle_sids', 'ellipse_sids', 'sids','center_point', 'cloud_internal', 'ellipse_trixels', 'circle_trixels']]

cells_small = cells[['x','y','sids_covers_19']]
r0_grid2019_small = r0['grid_2019'][mod09ga_bandnames + ['x', 'y']]
r0_grid_small = r0['grid'][mod09ga_bandnames + ['x', 'y']]

In [ ]:
merged = pandas.merge(mod09ga_small, mod09_small, how='left', 
                      left_on=mod09ga_bandnames,
                      right_on=mod09_bandnames,
                      suffixes=('', '_mod09'))

merged = pandas.merge(merged, cells_small, how='left',
                      left_on=['x', 'y'],
                      right_on=['x', 'y'],
                      suffixes=('', '_cell'))

merged = pandas.merge(merged, r0_grid2019_small,  how='left',
                      left_on=['x', 'y'],
                      right_on=['x', 'y'],
                      suffixes=['', '_r0_2019'])
    
merged = pandas.merge(merged, r0_grid_small,  how='left',
                      left_on=['x', 'y'],
                      right_on=['x', 'y'],
                      suffixes=['', '_r0_grid'])

merged.dropna(inplace=True)
#merged.drop_duplicates(mod09ga_bandnames, inplace=True)

In [ ]:
merged.info()

## Function definitions

In [ ]:
def find_ifov(ifovs, spectrum):
    if isinstance(spectrum, pandas.core.frame.DataFrame):   
        spectrum = [spectrum['sur_refl_b01_1'], spectrum['sur_refl_b02_1'], spectrum['sur_refl_b03_1'], spectrum['sur_refl_b04_1'], 
                    spectrum['sur_refl_b05_1'], spectrum['sur_refl_b06_1'], spectrum['sur_refl_b07_1']]
    rows = ifovs[(numpy.isclose(ifovs['500m Surface Reflectance Band 1'], spectrum[0])) & 
                 (numpy.isclose(ifovs['500m Surface Reflectance Band 2'], spectrum[1])) & 
                 (numpy.isclose(ifovs['500m Surface Reflectance Band 3'], spectrum[2])) & 
                 (numpy.isclose(ifovs['500m Surface Reflectance Band 4'], spectrum[3])) & 
                 (numpy.isclose(ifovs['500m Surface Reflectance Band 5'], spectrum[4])) & 
                 (numpy.isclose(ifovs['500m Surface Reflectance Band 6'], spectrum[5])) & 
                 (numpy.isclose(ifovs['500m Surface Reflectance Band 7'], spectrum[6])) ]
    return rows

def spectrum(df):    
    spectrum = df[['500m Surface Reflectance Band 1', 
                   '500m Surface Reflectance Band 2',
                   '500m Surface Reflectance Band 3',
                   '500m Surface Reflectance Band 4',
                   '500m Surface Reflectance Band 5',
                   '500m Surface Reflectance Band 6',
                   '500m Surface Reflectance Band 7']].to_numpy()[0]
    return spectrum

def vnp_spectrum(df):    
    spectrum = df[['750m Surface Reflectance Band M5', 
                   '750m Surface Reflectance Band M7',
                   '750m Surface Reflectance Band M3',
                   '750m Surface Reflectance Band M4',
                   '750m Surface Reflectance Band M8',
                   '750m Surface Reflectance Band M10',
                   '750m Surface Reflectance Band M11']].to_numpy()
    return spectrum

In [ ]:
def speedy_invert(lut, r, r0, solar_z):        
    result = {}    
    
    shade = numpy.zeros(7)    
    res, model_refl = spires.speedy_invert(lut, r, r0, solar_z, shade, mode=4)
    result['fsca'] = res.x[0]
    result['fshade'] = res.x[1]
    result['rg'] = res.x[2]
    result['dust'] = res.x[3]
    return result

def get_ground_truth(sids, worldview):    
    gt = starepandas.speedy_subset(worldview, sids)
    if gt.empty:
        return None
    else:
        return  gt['snow'].sum() / gt['snow'].count()    

In [ ]:
def compute_cells(row, worldview, lut):    
    solar_z = row.SolarZenith
    r = row[mod09_bandnames].to_numpy()
    sids_cell = row['sids_covers_19']
    sids_circle = row['circle_sids']
    sids_ellipse = row['ellipse_sids']
        
    r0_2019 = row[(s + '_r0_2019' for s in mod09ga_bandnames)].to_numpy()
    r0_grid = row[(s + '_r0_grid' for s in mod09ga_bandnames)].to_numpy()    
            
    res_2019 = speedy_invert(lut, r, r0_2019, solar_z)
    res_grid = speedy_invert(lut, r, r0_grid, solar_z)
        
    gt_circle = get_ground_truth(sids_circle, worldview) 
    gt_ellipse = get_ground_truth(sids_ellipse, worldview)
    gt_cell = get_ground_truth(sids_cell, worldview)
    
    results = {'2019': res_2019, 
               'grid': res_grid, 
               'gt_cell': gt_cell,
               'gt_circle': gt_circle, 
               'gt_ellipse': gt_ellipse}
    results = pandas.json_normalize(results, sep='_').iloc[0]
    return results

In [ ]:
def compute_stare(row, lut, r0_stare):       
    level = pystare.spatial_resolution(r0_stare.sids).max()    
    
    scan_pos = row.scan_pos
    solar_z = row.SolarZenith
    r = row[mod09_bandnames].to_numpy()
    
    row_sid = pystare.spatial_coerce_resolution(row.sids, level)
    row_sid = pystare.spatial_clear_to_resolution(row_sid)
        
    try:
        candidates = r0_stare[r0_stare.stare_intersects(row_sid)]                        
        r0_best = candidates.iloc[(candidates.scan_pos-scan_pos).abs().argsort().iloc[:1]]        
        r0_best = r0_best[mod09_bandnames].to_numpy()[0]        
    except:        
        print(row.x, row.y)        
        return pandas.DataFrame({'stare_fsca': [None], 'stare_fshade': [None], 'stare_rg': [None], 'stare_dust': None}).iloc[0]
        
    res_stare = speedy_invert(lut, r, r0_best, solar_z)
    
    results = {'stare': res_stare}    
    results = pandas.json_normalize(results, sep='_').iloc[0]
    return results

In [ ]:
def compute_vnp(row, lut, r0_stare, worldview):    
    level = pystare.spatial_resolution(r0_stare.sids).max()    
    sids_circle = row['circle_sids']
    
    scan_pos = row.scan_pos
    solar_z = row.solar_zenith
    r = row[vnp09_bandnames].to_numpy()
            
    row_sid = pystare.spatial_coerce_resolution(row.sids, level)
    row_sid = pystare.spatial_clear_to_resolution(row_sid)
        
    try:
        candidates = r0_stare[r0_stare.stare_intersects(row_sid)]                        
        r0_best = candidates.iloc[(candidates.scan_pos-scan_pos).abs().argsort().iloc[:1]]        
        try:
            r0_best = r0_best[vnp09_bandnames].to_numpy()[0]
        except:
            r0_best = r0_best[mod09_bandnames].to_numpy()[0]
    except:        
        print(row.index)        
        return pandas.DataFrame({'stare_fsca': [None], 'stare_fshade': [None], 'stare_rg': [None], 'stare_dust': None, 'gt': None}).iloc[0]
    
    gt_circle = get_ground_truth(sids_circle, worldview) 
        
    res_stare = speedy_invert(lut, r, r0_best, solar_z)
    
    results = {'stare': res_stare}    
    results = pandas.json_normalize(results, sep='_').iloc[0]
    results['gt'] = gt_circle
    return results    

## Execute SingleCore 

In [ ]:
%%time 
#s = merged[100:101].apply(compute_stare, axis=1, r0_stare=r0['stare14_1'], lut=lut)
#v = vnp09_day[0:1].apply(compute_vnp, axis=1, r0_stare=r0_vnp['vnp_stare15_1'], lut=lut, worldview=worldview_19)
v = vnp09_day[0:1].apply(compute_vnp, axis=1, r0_stare=r0['stare15_1'], lut=lut, worldview=worldview_19)
#c = merged[100:110].apply(compute_cells, axis=1, worldview=worldview_19, lut=lut)
v

## Dask
lsof -i :9895

Port forward to get to the dashboard

    ssh -N -L 8001:localhost:8787 schiss.eri.ucsb.edu

In [ ]:
cluster = dask.distributed.LocalCluster(n_workers=64, threads_per_worker=1, memory_limit='5GB', processes=True, dashboard_address='localhost:8787', silence_logs=logging.ERROR)

In [ ]:
ddf = dask.dataframe.from_pandas(merged, npartitions=512)
lut_delayed = dask.delayed(lut)
worldview_delayed = dask.delayed(worldview_19)

### Cells

In [ ]:
%%time
meta={'gt_cell':     numpy.float64,
      'gt_circle':   numpy.float64,
      'gt_ellipse':  numpy.float64,
      '2019_fsca':   numpy.float64,
      '2019_fshade': numpy.float64,
      '2019_rg' :    numpy.float64,
      '2019_dust':   numpy.float64,
      'grid_fsca':   numpy.float64,
      'grid_fshade': numpy.float64,
      'grid_rg':     numpy.float64,
      'grid_dust':   numpy.float64}

res = ddf.apply(compute_cells, axis=1, meta=meta, worldview=worldview_delayed, lut=lut_delayed)

with dask.distributed.Client(cluster, timeout=1200) as client:              
    fsca_cell = res.compute()

### MODIS

In [ ]:
meta = {'stare_fsca':   numpy.float64,
        'stare_fshade': numpy.float64,
        'stare_rg':     numpy.float64,
        'stare_dust':   numpy.float64}

ddf = dask.dataframe.from_pandas(merged, npartitions=1024)

def run(r0_string):    
    r0_stare = r0[r0_string]
    res = ddf.apply(compute_stare, axis=1, meta=meta, r0_stare=r0_stare, lut=lut_delayed)
    with dask.distributed.Client(cluster) as client:              
        res = res.compute()
    return res    

In [ ]:
%%time

fsca_stare_modis = {}
for r0_string in sorted(r0.keys()):
    print(r0_string)
    if 'stare' not in r0_string:
        continue        
    fsca_stare_modis[r0_string] = run(r0_string)

In [ ]:
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "monospace",
    "font.size"   : 13
})

In [ ]:
fig = plt.figure(figsize=(16, 9), dpi=100)
ax = fig.add_subplot()
plt.subplots_adjust(left=0.05, bottom=0.2, right=0.95, top=0.95, wspace=0, hspace=0)

fsca = pandas.DataFrame()

fsca['2019_vs_cell'] = fsca_cell['2019_fsca'] - fsca_cell['gt_cell']
fsca['2019_vs_circle'] = fsca_cell['2019_fsca'] - fsca_cell['gt_circle']
fsca['2019_vs_ellipse'] = fsca_cell['2019_fsca'] - fsca_cell['gt_ellipse']

#fsca['grid_vs_cell'] = fsca_cell['grid_fsca'] - fsca_cell['gt_cell']
#fsca['grid_vs_circle'] = fsca_cell['grid_fsca'] - fsca_cell['gt_circle']
#fsca['grid_vs_ellipse'] = fsca_cell['grid_fsca'] - fsca_cell['gt_ellipse']

r0s = sorted(r0.keys())#[20:]
#r0s = ['stare17_1']

for r0_string in r0s:
    if 'stare' not in r0_string:
        continue
    fsca[f'{r0_string}_vs_ellipse'] = fsca_stare_modis[r0_string]['stare_fsca'] - fsca_cell['gt_ellipse']

#fsca = fsca[(merged.ts_start>'2017-12-11') & (merged.ts_start<'2017-12-12') & (merged.y<575) & (merged.y>552)  & (merged.cloud_internal==False) & (merged.cloud<=1)]
#fsca = fsca[(merged.ts_start>'2017-12-07') & (merged.ts_start<'2017-12-08')  & (merged.cloud_internal==False) & (merged.cloud<=1)]
#fsca = fsca[(merged.ts_start>'2017-12-11') & (merged.ts_start<'2017-12-12')  & (merged.cloud_internal==False) & (merged.cloud<=1)]

fsca = fsca[(merged.cloud<=1) & (merged.cloud_internal==False) ]
fsca = fsca[['2019_vs_cell', '2019_vs_circle', '2019_vs_ellipse', 'stare14_1_vs_ellipse', 'stare14_4_vs_ellipse', 'stare15_4_vs_ellipse',  'stare16_4_vs_ellipse','stare17_4_vs_ellipse']]

#fsca = fsca.drop_duplicates('stare17_3_vs_ellipse')
#fsca = fsca[['2019_vs_cell',  '2019_vs_ellipse', 'stare16_1_vs_ellipse', 'stare16_4_vs_ellipse']]


fsca.boxplot(ax=ax, rot=0, showfliers=False)
plt.autoscale()
ax.set_xticks([1,2,3,4,5,6,7,8,], 
              ['ifov: Cell\n$R_0$:\t Grid\n$n_{view}$: 1', 
               'ifov: Circle\n$R_0$:\t Grid\n$n_{view}$: 1', 
               'ifov: Ellipse\n$R_0$:\t Grid\n$n_{view}$: 1', 
               'ifov: Ellipse\n$R_0$:\t L14\n$n_{view}$: 1',
               'ifov: Ellipse\n$R_0$:\t L14\n$n_{view}$: 4',               
               'ifov: Ellipse\n$R_0$:\t L15\n$n_{view}$: 4',               
               'ifov: Ellipse\n$R_0$:\t L16\n$n_{view}$: 4',               
               'ifov: Ellipse\n$R_0$:\t L17\n$n_{view}$: 4',], 
              horizontalalignment='left')
#ax.set_ylim(-0.1, 0.1)
fig.tight_layout()
fig.savefig('results.pdf')

In [161]:
pandas.DataFrame({'AME': fsca.abs().mean(),
                 'RMSE': numpy.sqrt(numpy.square(fsca).mean()),
                 'Variance': fsca.var()}).round(4)#.to_markdown('/home/griessbaum/m.md')#.sort_values('AME')

,AME,RMSE,Variance
2019_vs_cell,0.0617,0.0862,0.0070
2019_vs_ellipse,0.0523,0.0714,0.0047
stare16_1_vs_ellipse,0.0392,0.0608,0.0037
stare16_4_vs_ellipse,0.0373,0.0599,0.0036


### VNP

In [ ]:
ddf = dask.dataframe.from_pandas(vnp09_day, npartitions=512)
lut_delayed = dask.delayed(lut)
worldview_delayed = dask.delayed(worldview_19)

In [ ]:
fsca_stare = {}

In [ ]:
%%time

meta = {'stare_fsca':   numpy.float64,
        'stare_fshade': numpy.float64,
        'stare_rg':     numpy.float64,
        'stare_dust':   numpy.float64,
        'gt':           numpy.float64}

def run(r0_string):    
    r0_stare = r0_vnp[r0_string]
    res = ddf.apply(compute_vnp, axis=1, meta=meta, r0_stare=r0_stare, lut=lut_delayed, worldview=worldview_19)
    with dask.distributed.Client(cluster) as client:              
        res = res.compute()
    return res    
for r0_string in sorted(r0_vnp.keys()):
    print(r0_string)
    if 'stare' not in r0_string:
        continue        
    fsca_stare[r0_string] = run(r0_string)

In [ ]:
%%time

meta = {'stare_fsca':   numpy.float64,
        'stare_fshade': numpy.float64,
        'stare_rg':     numpy.float64,
        'stare_dust':   numpy.float64,
        'gt':           numpy.float64}

def run(r0_string):    
    r0_stare = r0[r0_string]
    res = ddf.apply(compute_vnp, axis=1, meta=meta, r0_stare=r0_stare, lut=lut_delayed, worldview=worldview_19)
    with dask.distributed.Client(cluster) as client:              
        res = res.compute()
    return res    

for r0_string in sorted(r0.keys()):
    print(r0_string)
    if 'stare' not in r0_string:
        continue        
    fsca_stare[r0_string] = run(r0_string)

## Closing and Plotting

In [73]:
#client.close()
cluster.close()

1331 605
1402 545
1364 581
1331 607
1330 608
1331 608
1316 614
1401 546
1327 611
1328 611
1329 611
1334 608
1401 546
1400 546
1332 608
1333 608
1334 608
1329 608
1330 608
1331 608
1332 608
1332 606
1337 606
1330 609
1332 609
1332 610
1329 610
1330 610
1331 610
1364 583
1329 611
1330 611
1403 544
1404 544
1400 546
1403 544
1404 544
1330 611
1330 609
1365 582
1389 547
1403 544
1404 544
1330 606
1331 606
1330 607
1331 607
1329 612
1364 583
1333 609
1327 610
1332 610
1361 587
1364 583
1400 546
1401 546
1329 610
1330 607
1332 607
1365 582
1401 546
1401 545
1402 545
1403 545
1405 543
1406 543
1331 609
1332 609
1328 610
1330 609
1331 609
1332 609
1333 609
1328 610
1329 610
1330 610
1331 610
1332 610
1330 607
1331 607
1330 609
1332 609
1329 609
1364 583
1331 608
1332 608
1401 544
1401 545
1332 607
1402 545
1401 545
1402 545
1403 545
1400 545
1401 545
1330 607
1330 609
1334 607
1332 604
1401 545
1374 568
1390 547
1331 605
1401 545
1402 545
1334 608
1401 545
1402 545
1329 609
1330 609
1333 609
1

In [ ]:
fig = plt.figure(figsize=(10, 8), dpi=150)
ax = fig.add_subplot()
plt.subplots_adjust(left=0.05, bottom=0.2, right=0.95, top=0.95, wspace=0, hspace=0)

df = pandas.concat(fsca_stare)
df = df['stare_fsca'] - df['gt']
df = df.reset_index()
df = pandas.pivot_table(df.reset_index(), index= 'level_1', values=0, columns = 'level_0')

df.boxplot(ax=ax, rot=90, showfliers=False)
#ax.set_ylim(-0.3, 0.3)
#fig.tight_layout()

In [ ]:
pandas.DataFrame({'AME': df.abs().mean(),
                 'RMSE': numpy.sqrt(numpy.square(df).mean()),
                 'Variance': df.var()}).round(4)#.to_markdown('/home/griessbaum/m.md')#
#df.abs().mean().sort_values()

In [ ]:
plt.close('all')

## Boxplots

# Visualizations

In [ ]:
ax = mod09ga[0:1].plot()
cells.set_trixels('trixels_covers_18')[0:1].plot(ax=ax)

In [ ]:
for idx, row in mod09ga.iterrows():    
    sids = pystare.spatial_coerce_resolution(row['sids'], 19)
    cell = cells[cells.set_sids('sids_covers_19').stare_intersects(sids)].to_dict()
    cell_sids = cell['sids_covers_19']#.iloc[0]
    print(row.x, row.y, cell['x'], cell['y'])

In [ ]:
ys = mod09ga['x'].to_numpy()#[0:64]
xs = mod09ga['y'].to_numpy()#[0:64]

# Ned's x = 1374; y = 568
- 15: 0.754448
- 16: 0.750294 
- 17: 0.749762

# Red's lake x = 1373; y = 566
- 15: 0.210236
- 15_7: 0.277106
- 16_5: 0.210236
- 17: 0.210236

# McCoy x = 1379; y = 565
- 15_5: 0.471604
- 15_7: 0.400707
- 16_1: 0.485828
- 16_5: 0.484894 
- 16_7: 0.431481
- 17_5: 0.484894

# Top Of Mammoth x = 1375; y = 568
- 15: 0.754448
- 16: 0.750294
- 17_5: 0.749762

In [ ]:
r0_stare = r0_stare16_7

In [ ]:
x=xs[50]
y=ys[50]
#x = 1374; y = 568
#x = 1373; y = 566
x = 1379; y = 565
#x = 1375; y = 568
compute(x,y)

In [ ]:
level

In [ ]:
mod09.set

In [ ]:
start = time.time()

with multiprocessing.Pool(processes=64) as pool:
    results = pool.starmap(compute, zip(xs, ys))

print(time.time()-start)

In [ ]:
results = pandas.concat(results)
results['cell_error_cellR0'] = results['gt_cell']- results['cell_fsca']
results['cirlce_error_cellR0'] = results['gt_circle']- results['cell_fsca']
results['ellipse_error_cellR0'] = results['gt_ellipse']- results['cell_fsca']

results['cell_error_stareR0'] = results['gt_cell']- results['stare_fsca']
results['cirlce_error_stareR0'] = results['gt_circle']- results['stare_fsca']
results['ellipse_error_stareR0'] = results['gt_ellipse']- results['stare_fsca']

In [ ]:
fig = plt.figure(figsize=(12, 7), dpi=100)
ax = fig.add_subplot()

results.boxplot(['cell_error_cellR0','cirlce_error_cellR0', 'ellipse_error_cellR0', 'cell_error_stareR0', 'cirlce_error_stareR0', 'ellipse_error_stareR0'])
ax.set_ylim(-0.3, 0.6)
fig.savefig('boxplot.png')

## Single cell

In [ ]:
# Reds Lake: 
#x = 1373; y = 566

# Ned's favorite / Kamikaze Run
x = 1374; y = 568

# Main Lodge
#x = 1380; y = 563

# McCoy / CUES
#x = 1379; y = 565

# Top Of Mammoth
#x = 1375; y = 568

In [ ]:
cell = cells[(cells.x==x) & (cells.y==y)]

r0_cell = r0_grid[(r0_grid.x==y) & (r0_grid.y==x)]
r0_cell = find_ifov(mod09, r0_cell)

mod09ga_cell = mod09ga[(mod09ga.x==y) & (mod09ga.y==x)]
mod09ga_cell = find_ifov(mod09, mod09ga_cell)

solar_z = mod09ga_cell['SolarZenith'].iloc[0]
shade = numpy.zeros(7)

In [ ]:
speedy_invert(r=mod09ga_cell, r0=r0_cell)

In [ ]:
r0_best = get_best_r0(mod09ga_cell)    
speedy_invert(mod09ga_cell, r0_best)

In [ ]:
cell_sids = cell['sids_covers_20'].iloc[0]
get_ground_truth(cell_sids)

In [ ]:
circle_sids = make_circle_sids(mod09ga_cell)
get_ground_truth(circle_sids)

In [ ]:
fig = plt.figure(figsize=(12, 7), dpi=100)
ax = fig.add_subplot()

circle = starepandas.STAREDataFrame(sids=[circle_sids])
circle.set_trixels(circle.make_trixels(), inplace=True)
circle.set_geometry('trixels', inplace=True)

mod09ga_cell.to_crs(epsg=3857).plot(ax=ax, trixels=False, color='blue', label='observation')
circle.to_crs(epsg=3857).plot(ax=ax, trixels=True, linewidth=0.2, color='blue', label='circular iFOV approximation')
r0_best.to_crs(epsg=3857).plot(ax=ax, trixels=False, color='cyan', label='best R0')
cell.set_geometry('polygons').exterior.to_crs(epsg=3857).plot(ax=ax, zorder=2, color='green', label='grid cell')
r0_cell.to_crs(epsg=3857).plot(ax=ax, trixels=False, color='red', label='gridded R0')

ax.legend()

contextily.add_basemap(ax, source=contextily.providers.OpenStreetMap.Mapnik)

In [ ]:
scan_pos

In [189]:
mod09[mod09['ts_start'] == '2021-01-09'>]

,ts_start,index,lat,lon,sids,ts_end,scan_pos,track_pos,500m Surface Reflectance Band 1,500m Surface Reflectance Band 2,...,sids14,sids15,sids16,sids17,sids18,center_point,area,level,ndsi,ndvi
